In [93]:
from requests import get,codes
from bs4 import BeautifulSoup
import time
import os
import pickle
import lzma
import urllib.parse
import numpy
import random
from fake_useragent import UserAgent
import tqdm

In [2]:
url = "https://free-proxy-list.net/"

In [3]:
proxy = get(url)

In [4]:
soup = BeautifulSoup(proxy.text, 'html.parser')

In [5]:
soup.find_all("tbody")[0].find_all("tr")[0].find_all("td")

[<td>217.182.120.162</td>,
 <td>1080</td>,
 <td>FR</td>,
 <td class="hm">France</td>,
 <td>elite proxy</td>,
 <td class="hm">no</td>,
 <td class="hx">yes</td>,
 <td class="hm">24 seconds ago</td>]

In [6]:
ip_addresses = []
for en in soup.find_all("tbody")[0].find_all("tr"):
    parsed_entry = en.find_all("td")
    if(parsed_entry[4].text == "elite proxy"):
        ip_addresses.append(parsed_entry[0].text+":"+parsed_entry[1].text)

proxy mess

In [23]:
ua = UserAgent()
base_url = "https://www.planespotters.net/"
url = "airlines/country"
base_path = os.path.join(".", "data","planespotter")

In [29]:
ua.random

'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:21.0) Gecko/20130331 Firefox/21.0'

In [96]:
def fetch_or_retrive(url_extension: str) -> (get, bool):
    """ For fetching content of webpage from "https://www.planespotters.net/".
    Will store local version of the page once fetched. New calls to this function
    will not create a new online requests
    
    :returns: tuple. first element is the request fetwched. The second is true is data found locally. 
    """
    
    folder_path = os.path.join(base_path, *url_extension.split('/')[:-1])
    file_path = os.path.join(folder_path, (url_extension.split('/')[-1]+".html.xz"))
    if(os.path.isfile(file_path)):
        with lzma.open(file_path, "rb") as wiki_file:
            print("loading data from hard drive")
            r = pickle.load(wiki_file)
            return r, True
    else:
        try:
            url =  urllib.parse.urljoin(base_url, url_extension)
            headers = {'User-Agent': ua.random}
            proxy_index = numpy.random.randint(0, len(ip_addresses) - 1)
            proxy = {"http": ip_addresses[proxy_index], "https": ip_addresses[proxy_index]}
            r = get(url, headers=headers) #faking the fact that we are not a bot.
            print("fetching online wikipedia page")
            print(r.status_code)
            if(r.status_code == codes.ok):
                os.makedirs(folder_path, exist_ok=True)
                with lzma.open(file_path, "wb") as wiki_file:
                    pickle.dump(r, wiki_file)
            else:
                raise Exception("Cannot find local version of data and website not responding with 200.")
        except:
            raise Exception("Cannot access the internet")
            
        return r, False

In [89]:
fetch_or_retrive("country/operators/Albania")

fetching online wikipedia page
200


<Response [200]>

In [78]:
import pandas as pd
country = pd.read_csv(os.path.join(".", "data","iso3166-countries-regional-codes.csv.xz"))

len(country[country.region == "Europe"].name.values)

51

In [97]:
countries_data = fetch_or_retrive("airlines/country")[0]

loading data from hard drive


In [80]:
dict_country = {}
soup = BeautifulSoup(countries_data.text, 'lxml')
coutry_list_raw = soup.find('div', attrs={'id': 'CountryList'})
for c in coutry_list_raw.find_all('a'):
    dict_country[c.text] = c.get('href')[1:]

In [81]:
temp = []
dict_european_country = {}
for d in dict_country.copy().items():
    if(d[0] in (country[country.region == "Europe"].name.values)):
        dict_european_country[d[0]] = d[1]

In [83]:
dict_european_country

{'Albania': 'country/operators/Albania',
 'Andorra': 'country/operators/Andorra',
 'Austria': 'country/operators/Austria',
 'Belarus': 'country/operators/Belarus',
 'Belgium': 'country/operators/Belgium',
 'Bulgaria': 'country/operators/Bulgaria',
 'Croatia': 'country/operators/Croatia',
 'Denmark': 'country/operators/Denmark',
 'Estonia': 'country/operators/Estonia',
 'Faroe Islands': 'country/operators/Faroe-Islands',
 'Finland': 'country/operators/Finland',
 'France': 'country/operators/France',
 'Germany': 'country/operators/Germany',
 'Gibraltar': 'country/operators/Gibraltar',
 'Greece': 'country/operators/Greece',
 'Guernsey': 'country/operators/Guernsey',
 'Hungary': 'country/operators/Hungary',
 'Iceland': 'country/operators/Iceland',
 'Ireland': 'country/operators/Ireland',
 'Isle of Man': 'country/operators/Isle-of-Man',
 'Italy': 'country/operators/Italy',
 'Jersey': 'country/operators/Jersey',
 'Latvia': 'country/operators/Latvia',
 'Liechtenstein': 'country/operators/Liec

In [84]:
len(dict_european_country)

44

In [ ]:

dict_response = {}
for country, url in dict_european_country.items():
    print(country + " " + url)
    dict_response[country], found_on_disk = fetch_or_retrive(url)
    if(not found_on_disk):
        time.sleep(60 + numpy.random.randint(0,40))


Albania country/operators/Albania
loading data from hard drive
Andorra country/operators/Andorra
fetching online wikipedia page
200
Austria country/operators/Austria
fetching online wikipedia page
200
Belarus country/operators/Belarus
fetching online wikipedia page
200
Belgium country/operators/Belgium
fetching online wikipedia page
200
Bulgaria country/operators/Bulgaria
fetching online wikipedia page
200
Croatia country/operators/Croatia
fetching online wikipedia page
200
Denmark country/operators/Denmark
fetching online wikipedia page
200


In [ ]:
dataframes = []
for table in soup.find_all("table"):
    table_headers = []
    dict_plane = {}
    for header in table.find_all("th"):
        head = header.text.replace("\n", "")
        table_headers.append(head)
        dict_plane[head] = []
    for line in table.find_all("tr"):
        for i, elem in enumerate(line.find_all('td')):
            dict_plane[table_headers[i]].append(elem.text.replace("\n", ""))
    dataframes.append(dict_plane)
    
dataframes = dataframes[:-1] #We don't want the private jets
print("There are " + str(len(dataframes))+ " dataframes")